## We finna wrangle

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import mason_functions as mf

In [2]:
def get_db_url(db_name):
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

### Exercise I
Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [3]:
#define my sql query into the relational database
sql = '''
SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
FROM properties_2017
WHERE propertylandusetypeid = 261 or propertylandusetypeid = 279
'''

#define my url
url = get_db_url('zillow')

#read the information from the db into a df
#I also don't want to keep querying the codeup rdbms (in case the kernel gotta go)
if os.path.isfile('properties_2017.csv'):
    df = pd.read_csv('properties_2017.csv', index_col = 0)
else:
    df = pd.read_sql(sql, url)
    df.to_csv('properties_2017.csv')

In [4]:
#at a glance
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152864 entries, 0 to 2152863
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 131.4 MB
None


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


### Exercise II
Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

In [5]:
#summary statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152853.0,3.287195,0.954757,0.00,3.0000,3.000,4.0000,25.00
bathroomcnt,2152853.0,2.230687,0.999281,0.00,2.0000,2.000,3.0000,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.0000,1623.000,2208.0000,952576.00
taxvaluedollarcnt,2152371.0,461896.052361,699675.940049,1.00,188170.0000,327671.000,534527.0000,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.0000,1958.000,1976.0000,2016.00
taxamount,2148422.0,5634.863752,8178.908996,1.85,2534.9725,4108.945,6414.3175,1337755.86
fips,2152864.0,6048.377330,20.433289,6037.00,6037.0000,6037.000,6059.0000,6111.00


In [6]:
#more perspective
df.sample(10)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
1804313,4.0,2.0,1505.0,439065.0,1947.0,5561.46,6037.0
831702,3.0,1.0,1341.0,39924.0,1931.0,569.18,6037.0
760734,2.0,2.0,1140.0,422000.0,1947.0,5788.19,6037.0
1750763,4.0,2.0,1284.0,510000.0,1957.0,6556.27,6037.0
174451,5.0,3.0,2081.0,321272.0,1965.0,4454.43,6037.0
1500205,6.0,5.0,4281.0,2030885.0,1994.0,23037.94,6037.0
942327,3.0,2.0,1660.0,353678.0,1957.0,4368.51,6037.0
1951568,3.0,1.0,1198.0,833734.0,1923.0,10089.92,6037.0
2115855,3.0,2.0,1355.0,291894.0,1968.0,3567.70,6037.0
865631,2.0,2.0,1783.0,784945.0,1940.0,8639.07,6037.0


In [7]:
#what kind of nulls am I looking at here?
df.isnull().any()

bedroomcnt                       True
bathroomcnt                      True
calculatedfinishedsquarefeet     True
taxvaluedollarcnt                True
yearbuilt                        True
taxamount                        True
fips                            False
dtype: bool

In [8]:
#sum of nulls by column
df.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8485
taxvaluedollarcnt                493
yearbuilt                       9338
taxamount                       4442
fips                               0
dtype: int64

In [9]:
#list the columns with null values
df.columns[df.isnull().any()]

Index(['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'yearbuilt', 'taxamount'],
      dtype='object')

In [10]:
#how many bedrooms can a single family residence have?
df.bedroomcnt.value_counts(dropna = False).sort_index(ascending = False)

25.0         1
18.0         3
16.0         2
15.0         6
14.0         7
13.0        16
12.0        12
11.0        34
10.0       121
9.0        291
8.0       1107
7.0       4807
6.0      25166
5.0     150866
4.0     634289
3.0     964298
2.0     335473
1.0      23166
0.0      13188
NaN         11
Name: bedroomcnt, dtype: int64

In [11]:
#how many bathrooms can a single family residence have?
df.bathroomcnt.value_counts(dropna = False).sort_index(ascending = False)

32.00         1
20.00         6
19.50         1
19.00         1
18.00         8
17.00         4
16.00        12
15.00        17
14.50         1
14.00        25
13.00        39
12.50         3
12.00        73
11.50         3
11.00       146
10.50        14
10.00       325
9.50         50
9.00        713
8.50        110
8.00       1692
7.50        384
7.00       4394
6.50       1333
6.00      10747
5.50       6217
5.00      28362
4.50      19506
4.00      82155
3.50      28518
3.00     422841
2.50     142981
2.00     943589
1.75          3
1.50      31211
1.00     414324
0.50         16
0.00      13028
NaN          11
Name: bathroomcnt, dtype: int64

In [12]:
#let's take a look at the rows where bedroom count is missing
df[df.bedroomcnt.isna()]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
107763,NaN,NaN,NaN,67366.0,1926.0,780.54,6059.0
118612,NaN,NaN,NaN,43992.0,1946.0,541.64,6059.0
193993,NaN,NaN,1348.0,840698.0,1952.0,NaN,6059.0
1141340,NaN,NaN,200.0,188972.0,NaN,NaN,6037.0
1324609,NaN,NaN,990.0,435000.0,1906.0,NaN,6037.0
1442976,NaN,NaN,NaN,273196.0,NaN,NaN,6037.0
1647347,NaN,NaN,400.0,28347.0,1954.0,NaN,6037.0
1701027,NaN,NaN,NaN,407930.0,1926.0,NaN,6037.0
1722708,NaN,NaN,NaN,477161.0,NaN,NaN,6037.0
1776423,NaN,NaN,NaN,38855.0,NaN,NaN,6037.0


In [13]:
#quick reference to number of observations
mf.add_commas(len(df))

'2,152,864'

In [14]:
#I really don't need 11 incomplete observations out of 2 million
df = df[df.bathroomcnt.notna()]

In [15]:
#just to be sure
df[df.bathroomcnt.isna()]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips


In [16]:
#just to be really sure
mf.add_commas(len(df))

'2,152,853'

In [17]:
#this is a 'triple check' just to make sure the bedroom count nulls went with the bathroom nulls
df[df.bedroomcnt.isna()]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips


In [18]:
#let's look at tax value dollar count rows where the dollar count is missing
df[df.taxvaluedollarcnt.isna()].sample(10)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
1862423,0.0,0.0,NaN,NaN,NaN,NaN,6111.0
345121,0.0,0.0,NaN,NaN,NaN,NaN,6111.0
1173628,0.0,0.0,NaN,NaN,NaN,NaN,6037.0
295343,0.0,0.0,NaN,NaN,NaN,NaN,6037.0
774706,0.0,0.0,NaN,NaN,NaN,NaN,6111.0
1195214,0.0,0.0,NaN,NaN,NaN,NaN,6037.0
1668643,0.0,0.0,NaN,NaN,NaN,NaN,6037.0
1964310,2.0,1.0,884.0,NaN,1923.0,NaN,6037.0
1765718,0.0,0.0,NaN,NaN,NaN,NaN,6111.0
1981026,0.0,0.0,NaN,NaN,1985.0,NaN,6111.0


In [19]:
#after cycling through the .sample function, I feel confident in dropping these rows as well, because in the 
# rows where the tax value is missing, there tend to be many other missing values
df = df[df.taxvaluedollarcnt.notna()]

In [20]:
#looks good
df[df.taxvaluedollarcnt.isna()]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips


In [21]:
#let's look at rows where the tax amount is missing
df[df.taxamount.isna()].sample(10)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
1406336,1.0,1.0,962.0,17461.0,1936.0,NaN,6037.0
660526,2.0,2.0,1656.0,89272.0,1922.0,NaN,6037.0
1587398,2.0,1.0,1260.0,158413.0,1916.0,NaN,6037.0
1054834,0.0,0.0,NaN,94794.0,NaN,NaN,6059.0
1908567,2.0,2.0,1758.0,2404410.0,1951.0,NaN,6037.0
1047325,2.0,1.0,837.0,231947.0,1956.0,NaN,6037.0
680908,3.0,2.0,1485.0,63139.0,1965.0,NaN,6037.0
855555,0.0,0.0,NaN,25519.0,NaN,NaN,6037.0
2028838,2.0,1.0,832.0,6263.0,1938.0,NaN,6037.0
1428437,3.0,2.0,1384.0,341441.0,1946.0,NaN,6037.0


In [22]:
#establish ratio between tax value dollar count and tax amount for all rows
df['tax_value_amount_ratio'] = df.taxvaluedollarcnt / df.taxamount

In [23]:
#get a feel for this new feature
df.tax_value_amount_ratio.describe().T

count    2.148389e+06
mean     7.998171e+01
std      8.377490e+01
min      3.219043e-03
25%      7.370649e+01
50%      8.072719e+01
75%      8.526636e+01
max      3.665118e+04
Name: tax_value_amount_ratio, dtype: float64

In [24]:
#no. of observations at this point in time
mf.add_commas(len(df))

'2,152,360'

In [30]:
df[(df.taxamount.notna()) & (df.taxvaluedollarcnt.notna())].sample(10)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,tax_value_amount_ratio
2010936,3.0,1.5,1049.0,234994.0,1961.0,2927.22,6059.0,80.278899
851489,2.0,2.0,1891.0,461938.0,1930.0,5806.13,6037.0,79.560396
2134204,4.0,2.0,1498.0,535716.0,1956.0,6254.94,6037.0,85.646865
2051476,2.0,1.0,780.0,645000.0,1950.0,7936.53,6037.0,81.269774
197205,1.0,3.0,2784.0,7475366.0,1958.0,84648.59,6037.0,88.310579
765699,6.0,3.0,1947.0,359330.0,1946.0,4491.02,6037.0,80.010777
1028940,2.0,1.0,1025.0,254647.0,1951.0,3401.14,6037.0,74.871073
591645,4.0,2.5,2166.0,665272.0,1993.0,6873.54,6059.0,96.787390
1100554,4.0,2.0,1343.0,149000.0,1953.0,2687.68,6037.0,55.438147
701601,3.0,2.0,1976.0,618176.0,1948.0,6865.95,6037.0,90.035028


In [31]:
df.fips.value_counts()
#6037 = Los Angeles County
#6059 = Orange County
#6111 = Ventura County
#all zillow single family residential properties in California? 

6037.0    1431486
6059.0     555051
6111.0     165823
Name: fips, dtype: int64

In [36]:
los_angeles_county = df[df.fips == 6037]
orange_county = df[df.fips == 6059]
ventura_county = df[df.fips == 6111]

In [41]:
los_angeles_county.taxvaluedollarcnt.describe()

count    1.431486e+06
mean     4.377894e+05
std      7.286610e+05
min      1.000000e+00
25%      1.698552e+05
50%      2.947320e+05
75%      4.868008e+05
max      9.018846e+07
Name: taxvaluedollarcnt, dtype: float64

In [ ]:
mf.add_commas(len(df))

### Exercise III
Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.